In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../../")

from src.traffic_predictor.config import ModelConfig, TrainingConfig, DatasetConfig
from src.DatasetManager import DatasetConvertor 
from src.traffic_predictor.preprocessing import PreparingDataset
#from src.traffic_predictor.models.ContextAssisted import TrafficPredictorContextAssisted as TrafficPredictor
from src.traffic_predictor.models.ContextAssisted import createModel
from src.traffic_predictor.training import trainModel
from src.traffic_predictor.evaluation import evaluateModel

In [3]:
rawDatasetFolder = "../../data/raw"

verbose = True
mapping = {"forward": "fr", "backward": "bk"}
dpdr_params = [
    {"dbParameter": 0.01, "alpha": 0.01, "mode": "fixed"}
]

direction = "forward"
datasetConverter = DatasetConvertor(rawDatasetFolder)
datasetConverter.processDataset(direction=direction, **dpdr_params[0])
thumbDataUnit = datasetConverter.getDataUnit(f"thumb_{mapping[direction]}")
lenWindow = 30
datasetConfig = DatasetConfig.initialize(len_window=lenWindow, data_augment=True)
trainData, testData = PreparingDataset(thumbDataUnit, datasetConfig, verbose=False)
(sources, _, lastTranmittedContext, _, _, _, _, _) = trainData
print(sources.shape)
print(lastTranmittedContext.shape)


========== Thumb ============
Forward: Compression rate: 0.2231
========== Index ============
Forward: Compression rate: 0.2456
========== Middle ============
Forward: Compression rate: 0.2491
(62380, 30, 3)
(62380, 1, 3)


In [4]:
modelConfig = ModelConfig.from_dataset(datasetConfig, trainData)
model, device = createModel(modelConfig)

trainingConfig = TrainingConfig()
best_model, avg_train_loss_history, avg_test_loss_history = trainModel(
    model, trainData, testData, trainingConfig, verbose=verbose
)

Size of train loader: 8, Size of test loader: 6
Used device: cuda:0
Size of model: 50621
TrafficPredictorContextAssisted(
  (dbf2traffic): DeadFeaturesToTrafficLayer(
    (input_layer): Sequential(
      (0): Linear(in_features=496, out_features=64, bias=True)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.8, inplace=False)
    )
    (hidden_layers): ModuleList(
      (0-2): 3 x Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Dropout(p=0.8, inplace=False)
      )
    )
    (trans2transmission_layer): Linear(in_features=64, out_features=30, bias=True)
    (sigmoid): Sigmoid()
    (trans2traffic_layer): Linear(in_features=94, out_features=1, bias=True)
    (trans2trafficClass_layer): Linear(in_features=94, out_features=31, bias=True)
  )
  (contextAdjus

In [ ]:
datasetConfig = DatasetConfig.initialize(len_window=lenWindow, data_augment=False)
_, testData = PreparingDataset(thumbDataUnit, datasetConfig, verbose=False)
traffic_predictor, device = createModel(modelConfig)
traffic_predictor.load_state_dict(best_model)
results = evaluateModel(traffic_predictor, testData)

plt.plot(results['trafficTarget_actual'][0:200])
plt.plot(results['trafficTarget_predicted'][0:200])
plt.show()